In [2]:
from git import Repo
# For cloning github repo

from langchain.text_splitter import Language
# Required for context aware splitting. Splitting the code into chunks. Each block of function or class is split into chunks of code
# Eg: def main() --> code inside the func is split into chunks of code

from langchain.document_loaders.generic import GenericLoader
# Inorder to load the github repository, genericloader is required

from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

### Cloning the repository

In [3]:
repo_path = "test_repo/"
Repo.clone_from("https://github.com/Sainivedhana/llama", to_path=repo_path)

<git.repo.base.Repo 'd:\\genai_llama\\Source-code-analysis-openAI\\research\\test_repo\\.git'>